In [1]:
from openai import OpenAI
import json
import inspect

In [2]:
with open('configs.json', 'r') as file:
    configs=json.load(file)

In [3]:
client = OpenAI(api_key = configs['open_ai_key'])

In [5]:
# Define the system prompt
system_prompt = """
You are a customer service assistant. 
1. Classify the user's question into one of the following categories: shipping, malfunction, billing, or other. 
2. Then, call the corresponding function to handle the query.
"""

In [6]:
def handle_shipping(query):
    print(f"Your question is related to shipping: {query}")

def handle_malfunction(query):
    print(f"Your question is related to malfunction: {query}")

def handle_billing(query):
    print(f"Your question is related to billing: {query}")

def handle_other(query):
    print(f"Your question is related to unknown topics")

In [7]:
def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

In [8]:
tools = [handle_shipping, handle_malfunction, handle_billing, handle_other]
tool_schemas = [function_to_schema(tool) for tool in tools]
tools_map = {tool.__name__: tool for tool in tools}

In [9]:
def call_agent(system_prompt, user_prompt, tool_schemas):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt},
                     {"role":"user","content":user_prompt}],
            tools=tool_schemas or None,
        )
    message = response.choices[0].message
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        tools_map[name](**args)

In [ ]:
def see_response(system_prompt, user_prompt, tool_schemas):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt},
                     {"role":"user","content":user_prompt}],
            tools=tool_schemas or None,
        )
    return response

In [10]:
prompt = input("Enter your query: ")
call_agent(system_prompt, prompt, tool_schemas)

Enter your query:  What happened with my gaming console I just bought yesterday and it broke


Your question is related to malfunction: My gaming console I just bought yesterday broke.
